# Assignment 2

## Part 1: Image Processing Basics

Computers use tiny dots called _pixels_ to display images. Each pixel is stored as an array of numbers that represent color intensities.

__Example.__ In an 8-bit grayscale image, each pixel is a single number. The number represents light intensity ranging from black (0) to white (255).

__Example.__ In a 24-bit RGB color image, each pixel is an array of 3 numbers. These numbers range from 0 to 255 and represent red, green, and blue intensity, respectively. For instance, `(0, 0, 255)` is <span style="color:#00F">bright blue</span> and `(255, 128, 0)` is <span style="color:#FF8000">orange</span>.

In this assignment, you'll use Python and NumPy to manipulate 24-bit RGB color images.

You can use `Image.open()` from the Python imaging library (PIL) to open an image:

In [9]:
from PIL import Image

# Cat image from https://unsplash.com/photos/FqkBXo2Nkq0
cat_img = Image.open("cat.png")

Images display inline in Jupyter notebooks:

In [24]:
cat_img.show()

In a Python terminal, you can display the image in a new window with `.show()` instead.

NumPy can convert images to arrays:

In [10]:
import numpy as np

cat = np.array(cat_img)

To convert an array back to an image (for display) use the function below:

In [12]:
def as_image(x):
    """Convert an ndarray to an Image.
    
    Args:
        x (ndarray): The array of pixels.
        
    Returns:
        Image: The Image object.
    """
    return Image.fromarray(np.uint8(x))

__Exercise 1.1.__ How many dimensions does the `cat` array have? What does each dimension represent?

In [13]:
cat.shape

(267, 400, 3)

__Exercise 1.2.__ Use `.copy()` to copy the cat array to a new variable. Swap the green and blue color channels in the copy. Display the result.

In [13]:
import copy
newcat = copy.copy(cat)
img = newcat[:,:,::-1]
img = as_image(img)
img.show()

__Exercise 1.3.__ Why is `.copy()` necessary in exercise 1.2? What happens if you don't use `.copy()`?

In [ ]:
you lose the original channel values for the color you swap if you do not use .copy()

__Exercise 1.4.__ Flip the blue color channel from left to right. Display the resulting image. _Hint: see the NumPy documentation on array manipulation routines._

In [14]:
newcat[:, :, 1] = np.fliplr(newcat[:, :, 1])
newcat = as_image(newcat)
newcat.show()

## Part 2: Singular Value Decomposition

Suppose $X$ is an $n \times p$ matrix (for instance, one color channel of the cat image). The _singular value decomposition_ (SVD) factors $X$ as $X = UD V^T$, where:

* $U$ is an $n \times n$ orthogonal matrix
* $D$ is an $n \times p$ matrix with zeroes everywhere except the diagonal
* $V$ is an $p \times p$ orthogonal matrix

Note that a matrix $A$ is _orthogonal_ when $A^T A = I$ and $AA^T = I$.

__Example.__ We can use NumPy to compute the SVD for a matrix:

In [15]:
x = np.array(
    [[0, 2, 3],
     [3, 2, 1]]
)
u, d, vt = np.linalg.svd(x)
# Here d is 2x2 because NumPy only returns the diagonal of D.
print "u is:\n", u, "\nd is:\n", d, "\nv^T is:\n", vt

u is:
[[-0.68145174 -0.73186305]
 [-0.73186305  0.68145174]] 
d is:
[ 4.52966162  2.54600974] 
v^T is:
[[-0.48471372 -0.62402665 -0.6128975 ]
 [ 0.80296442 -0.03960025 -0.59470998]
 [ 0.34684399 -0.78039897  0.52026598]]


If we let

* $u_i$ denote the $i$th column of $U$
* $d_i$ denote the $i$th diagonal element of $D$
* $v_i$ denote the $i$th column of $V$

then we can write the SVD as $\ X = UDV^T = d_1 u_1 v_1^T + \ldots + d_m u_m v_m^T\ $ using the rules of matrix multiplication. In other words, the SVD decomposes $X$ into a sum!

If we eliminate some of the terms in the sum, we get a simple approximation for $X$. For instance, we could eliminate all but first 3 terms to get the approximation $X \approx d_1 u_1 v_1^T + d_2 u_2 v_2^T + d_3 u_3 v_3^T$. This is the same as if we:

* Zero all but the first 3 diagonal elements of $D$ to get $D_3$, then compute $X \approx UD_3V^T$
* Eliminate all but the first 3 columns of $V$ to get $p \times 3$ matrix $V_3$, then compute $X \approx UDV_3^T$

We always eliminate terms starting from the end rather than the beginning, because these terms contribute the least to $X$.

Why would we want to approximate a matrix $X$?

In statistics, _principal components analysis_ uses this approximation to reduce the dimension (number of covariates) in a  centered (mean 0) data set. The vectors $d_i u_i$ are called the _principal components_ of $X$. The vectors $v_i^T$ are called the _basis vectors_. Note that both depend on $X$. The dimension is reduced by using the first $q$ principal components instead of the original $p$ covariates. In other words, the $n \times p$ data $X$ is replaced by the $n \times q$ data $UD_q = XV_q$

In computing, this approximation is sometimes used to reduce the number of bits needed to store a matrix (or image). If $q$ terms are kept, then only $nq + pq$ values (for $XV_q$ and $V_q^T$) need to be stored instead of the uncompressed $np$ values.

__Exercise 2.1.__ Write the functions described below.

* A function that takes a matrix $X$ and returns its principal component matrix $XV_q$ and basis matrix $V_q^T$. This function should also take the number of terms kept $q$ as an argument.

* A function that takes a principal component matrix $XV_q$ and basis matrix $V_q^T$ and returns an approximation $\hat{X}$ for the original matrix.

As usual, make sure to document your functions. Test your function on the red color channel of the cat image. What's the smallest number of terms where the cat is still recognizable as a cat?

In [27]:
othercat = copy.copy(cat)
red = othercat[:, :, 0]
def fun(x):
    q = input("Enter a value for q: ")
    u, d, vt = np.linalg.svd(x)
    v = np.transpose(vt)
    pcm = x.dot(v[:,q])
    est = (v[:,q]).dot(vt[:,q])
    print pcm.dot(est)
    print "Principal component matrix:" ,pcm
    print "Basis matrix:" ,vt[:,q]

fun(red)
    

Enter a value for q: 10
 [  4.28548117e+00   3.84108386e+00   2.96496272e+00   1.95107953e+00
   1.12247468e+00   5.97395584e-01   3.09774739e-01   1.10271425e-01
  -5.58213346e-02  -1.66379249e-01  -3.28704032e-01  -5.43349525e-01
  -8.37241244e-01  -1.06804679e+00  -1.24394559e+00  -1.33004377e+00
  -1.50461404e+00  -1.43002991e+00  -1.49177686e+00  -1.68477324e+00
  -1.91634661e+00  -2.03822870e+00  -1.97009788e+00  -1.83239590e+00
  -1.70789464e+00  -1.67454013e+00  -1.64390135e+00  -1.62776049e+00
  -1.62050643e+00  -1.46646573e+00  -1.30000320e+00  -1.14690684e+00
  -1.01310523e+00  -9.38791680e-01  -9.29416059e-01  -8.51842703e-01
  -6.39001656e-01  -2.94753162e-01  -7.21705560e-02   2.53397644e-01
   8.90105005e-01   1.39677711e+00   1.63754625e+00   2.02518556e+00
   2.39076180e+00   2.69754693e+00   2.69826555e+00   3.00661833e+00
   3.26895958e+00   3.63276605e+00   3.65367237e+00   3.47986296e+00
   3.21112381e+00   3.05659969e+00   2.06912970e+00   1.90202153e+00
   1.8544

__Exercise 2.2.__ You can check the number of bytes used by a NumPy array with the `.nbytes` attribute. How many bytes does the red color channel of the cat image use? How many bytes does the compressed version use when 10 terms are kept? What percentage of the original size is this?

In [49]:
red.nbytes

106800

In [ ]:
def fun(x):
    q = input("Enter a value for q: ")
    u, d, vt = np.linalg.svd(x)
    v = np.transpose(vt)
    pcm = x.dot(v[:,q])
    print "Principal component matrix:" ,pcm
    print "Basis matrix:" ,vt[:,q]
    print "Number of bytes:", pcm.nbytes
    
fun(red)


In [ ]:
print np.divide(2136.0,106800.0)
print "2 percent of original"